# Task: Resume Matching with Job Descriptions Using PDF CVs

Installing the required libraries

In [ ]:
!pip install PyPDF2

In [ ]:
!pip install python-docx
!pip install docx

In [ ]:
!pip install nltk
!pip install spacy==2.3.5
!pip install https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-2.3.1/en_core_web_sm-2.3.1.tar.gz
!pip install pyresparser

In [4]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

Loading Pretrained model for resume extraction and other libraries

In [5]:
import PyPDF2
from pyresparser import ResumeParser
import spacy
import re
import os
import pandas as pd



#1.Buildiing PDF Extractor


*   since the pre defined model is only good at extracting skills i have only extracted skills and job role.
*   For the extraction of the Education i have  used other pre trained model


**Function to Extract Job role and Skills**

In [ ]:

def extract_skills_job_from_pdf(pdf_path):
   text = ""
   with open(pdf_path, 'rb') as pdf_file:
       pdf_reader = PyPDF2.PdfReader(pdf_file)
       for page_num in range(len(pdf_reader.pages)):
           page = pdf_reader.pages[page_num]
           text += page.extract_text()
       words = text.split('\n')  # Split the text into words
       # Extracting the first word as the job role
       if words:
          category = words[0]
       else:
          category = None
       # Extracting the "Skills" section using resume parser
       data = ResumeParser(pdf_path).get_extracted_data()
       skills_str = ''
       for st in data['skills']:
          skills_str += ','+st
          skills_str = skills_str.strip()
       #checking if skills extracted are empty
       if len(skills_str) == 0:
        # if empty then trying to extract with regex pattern
         skills_pattern = r"Skills\n(.*?)(?=\n\S+\s*$|\Z)"
         skills_match = re.search(skills_pattern, text, re.DOTALL)
         if skills_match:
             skills_str = skills_str.strip()
   return category,skills_str



*   Extracting skills and job role and creating a dataset with it.




In [ ]:
# Specifying the folder containing the  CV_PDF files
root_dir = "/content/drive/MyDrive/data"

# Creating an empty DataFrame
pdf_data = pd.DataFrame(columns=["PDF Name", "Job role","skills"])

# Iterating through all PDF files in the folder
for folder_path, _, file_names in os.walk(root_dir):
    for file_name in file_names:
       if file_name.endswith(".pdf"):
           pdf_file_path = os.path.join(folder_path, file_name)
           job,skillss= extract_skills_job_from_pdf(pdf_file_path)

           # Replacing newline characters and commas in the content
           skillss = skillss.replace("\n", " ").replace(",", " ")

           # Append data to the DataFrame
           pdf_data = pdf_data.append({"PDF Name": file_name, "Job role": job,"skills":skillss}, ignore_index=True)

dataset_file = "CV_Extracted_skills.csv"

# Saving the DataFrame as a CSV file
pdf_data.to_csv(dataset_file, index=False)

print("PDF contents extracted and saved to the dataset.")




*   Displaying the created dataset




In [ ]:
#Loading the skills Dataset extracted above
df = pd.read_csv('/content/CV_Extracted_skills.csv')
df.head()


,PDF Name,Job role,skills
0,15576950.pdf,HR SPECIALIST,Policies Requests Data entry Inventory Proces...
1,14640322.pdf,HR GENERALIST,Audit Writing Policies Process Documentation ...
2,15575117.pdf,HR SENIOR SPECIALIST,Schedules Tax Policies Call center Requests D...
3,16877897.pdf,ASSISTANT MANAGER - HR,Audit Safety Distribution Process Payroll Sch...
4,11592605.pdf,HR GENERALIST,Proposal Requests Psychology Legal Regulatory...


In [ ]:
len(df)

2493





*   **Extracting Eucation part**







*   For the secong pretrained model spacy version 3.5.0 is required.






In [ ]:
!pip install spacy==3.5.0

In [ ]:
!pip install spacy_transformers

**Restart the Runtime in order to use installed spacy 3.5.0**

In [4]:
import spacy
from spacy.tokens import DocBin
import tqdm
import spacy_transformers

In [5]:
import pandas as pd
import os
import PyPDF2
import re


*   Loading the pretrained Model




This model is large to share via Github.so, i am providing the Google drive link:
https://drive.google.com/drive/folders/10l5qdmqYdqaCFnSPSxoRgYK9luQGWpjM?usp=drive_link

In [ ]:
#Loading the model with its directory
nlp = spacy.load("/content/drive/MyDrive/model-best")



*   **Function to extract Education part**




In [ ]:
def extract_education(pdf_path):
  text = ""
  with open(pdf_path, 'rb') as pdf_file:
      pdf_reader = PyPDF2.PdfReader(pdf_file)
      for page_num in range(len(pdf_reader.pages)):
          page = pdf_reader.pages[page_num]
          text += page.extract_text()
          doc = nlp(text)
          degree = ''
          institution = ''
          for ent in doc.ents:
              if ent.label_ == 'DEGREE':
                    degree +=','+ent.text
              if ent.label_ == 'COLLEGE NAME':
                    institution += ','+ent.text
      #combining degree and college into education
      education_str = institution+' '+degree
      education_str = education_str.replace("\n", " ").replace(",", " ")
      #Removing the numbers
      education_str = re.sub(r'\d+', '', education_str)
      education_str = education_str.strip()
      return education_str



*   **Extracting Education part**
*   This process takes some hours to complete,because there are more CV pdfs to extract.


In [ ]:
# Specifying the folder containing the  CV_PDF files
root_dir = "/content/drive/MyDrive/data"

# Creating an empty DataFrame
pdf_data1 = pd.DataFrame(columns=["PDF Name",'Education'])
# Iterating through all PDF files in the folder
for folder_path, _, file_names in os.walk(root_dir):
    for file_name in file_names:
       if file_name.endswith(".pdf"):
               pdf_file_path = os.path.join(folder_path, file_name)
               educationn= extract_education(pdf_file_path)

               # Replacing newline characters and commas in the content
               educationn = educationn.replace("\n", " ").replace(",", " ")

               # Append data to the DataFrame
               pdf_data1 = pdf_data1.append({"PDF Name": file_name, "Education":educationn}, ignore_index=True)

# Specifying the path to save the dataset CSV file
dataset_file = "CV_Extracted_edu.csv"

# Saving the DataFrame as a CSV file
pdf_data1.to_csv(dataset_file, index=False)

print("PDF contents extracted and saved to the dataset.")




*   It is taking more time to extract from full dataset. So,i have extracted from upto 1500 CVs.



In [22]:
#loading the dataset extracted and saved above.
CV_Extracted_edu = pd.read_csv("/content/CV_Extracted_edu.csv")

In [23]:
len(CV_Extracted_edu)

1568

In [9]:
CV_Extracted_edu = CV_Extracted_edu.drop_duplicates(subset=['PDF Name'])

In [15]:
dataset_file = 'CV_Extracted_edu.csv'
CV_Extracted_edu.to_csv(dataset_file, index=False)


*   Combining both extracted Skills and Education.

In [16]:
df1 = pd.read_csv("/content/CV_Extracted_skills.csv") #Dataset of Extracted skills
df2 = pd.read_csv("/content/CV_Extracted_edu.csv")#Dataset of Extracted Education
# Merging the two DataFrames based on the 'KeyColumn'
# The both Datasets are merged based on PDF name
full_cv_extracted = df1.merge(df2, on='PDF Name', how='left')

# Renaming the columns
full_cv_extracted = full_cv_extracted.rename(columns={'1st dataset': '1st_dataset', '2nd dataset': '2nd_dataset'})




In [18]:
full_cv_extracted.dropna(inplace=True)
len(full_cv_extracted)

1311



*   Saving the Extracted CV details

In [21]:
dataset_file = 'CV_Extracted_full.csv'
full_cv_extracted.to_csv(dataset_file, index=False)




*   Displaying the extracted CV deatails




In [30]:
full_cv_extracted.head(10)

,PDF Name,Job role,skills,Education
0,15576950.pdf,HR SPECIALIST,Policies Requests Data entry Inventory Proces...,Diploma
1,14640322.pdf,HR GENERALIST,Audit Writing Policies Process Documentation ...,M.B.A Bachelor of Science : Physics Honours...
3,16877897.pdf,ASSISTANT MANAGER - HR,Audit Safety Distribution Process Payroll Sch...,Bachelor of Education (B.Ed) MBA : Human Re...
5,15041689.pdf,HR DIRECTOR,Safety Tv Policies Hris Process Reconciliatio...,Newberry College H.S. Diploma Honor graduate ...
6,17855844.pdf,HR MANAGER,Policies Hris Legal Process Investigations Ch...,Interests B.S Member of Society of Human Reso...
7,18297650.pdf,VOLUNTEER HR -IVOLUNTEER,Schedules C++ Policies International Process ...,Post Graduate Diploma Masters Bachelor of Comm...
8,11847784.pdf,HR SPECIALIST,Hospital Policies Requests Legal Process Nego...,Bachelor of Arts : English Lit/Language
9,18731098.pdf,SENIOR HR MANAGER,Safety Tax Distribution Policies Hris Data en...,Master of Science : Health Administration B...
10,13520837.pdf,HR REPRESENTATIVE,Policies Hris Data entry Process Scheduling P...,Bachelor of Science
11,14225422.pdf,HR MANAGER/GENERALIST,Safety Proposal Metrics Policies Billing Hris...,Master of Science Degree Bachelor of Arts Degree




*   Upto now Extraction Part from CV dataset is completed.




# 2.Job Description Data Understanding

In [ ]:
!pip install transformers datasets

In [ ]:
!pip install transformers



*   Importing Job_description Dataset




In [26]:
from datasets import load_dataset

# Defining the dataset name to use from Hugging Face Hub
dataset_name = "jacob-hugging-face/job-descriptions"

# Load the dataset
dataset = load_dataset(dataset_name)

dataset

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['company_name', 'job_description', 'position_title', 'description_length', 'model_response'],
        num_rows: 853
    })
})



*   **Fetching from the Dataset**






*   Fetching the Desired job Descriptions from the dataset with position_titles.
*   Creating a new dataset with the filtered job descriptions.



In [27]:
# Listing of desired position titles
desired_position_titles = ["Sales leader","Regional Vice President of Sales","Manager, Business Strategy And Operations","Sales Manager","Retail Sales Associate","Registered Nurse","Project Manager","IT Project Manager","Sales And Marketing Specialist","Human Resources Director","Human Resources Manager","Senior Financial Analyst"	,"Accountant","Business Development Specialist","Sr Manager - Business Development"]
# Initializing a list to store filtered data
filtered_data_list = []

# Loop through the desired position titles and filter the dataset for each title
for desired_position_title in desired_position_titles:
    filtered_data = dataset['train'].filter(lambda example: example['position_title'] == desired_position_title)

    if len(filtered_data) > 0:
        filtered_data_list.append(filtered_data)

# Concatenating the filtered datasets into one
filtered_job_data = pd.concat([pd.DataFrame(filtered_data) for filtered_data in filtered_data_list])

# Save the filtered data to a new CSV file
#filtered_job_data.to_csv('filtered_job_descriptions.csv', index=False)


Filter:   0%|          | 0/853 [00:00<?, ? examples/s]

Filter:   0%|          | 0/853 [00:00<?, ? examples/s]

Filter:   0%|          | 0/853 [00:00<?, ? examples/s]

Filter:   0%|          | 0/853 [00:00<?, ? examples/s]

Filter:   0%|          | 0/853 [00:00<?, ? examples/s]

...

Filter:   0%|          | 0/853 [00:00<?, ? examples/s]

Filter:   0%|          | 0/853 [00:00<?, ? examples/s]

Filter:   0%|          | 0/853 [00:00<?, ? examples/s]

Filter:   0%|          | 0/853 [00:00<?, ? examples/s]

Filter:   0%|          | 0/853 [00:00<?, ? examples/s]

In [28]:
filtered_job_data = filtered_job_data.drop_duplicates(subset=['position_title'])

In [31]:
#removing the Duplicate index
filtered_job_data = filtered_job_data.reset_index(drop=True)


In [56]:
#Displaying the filtered dataset
filtered_job_data

,company_name,job_description,position_title,description_length,model_response
0,Gallin Associates,hybrid onsite remote for quasilocal candidate...,Sales leader,748,"{\n ""Core Responsibilities"": ""Manage a team ..."
1,Fisher Investments,calling all financial advisors looking for a r...,Regional Vice President of Sales,2901,"{\n ""Core Responsibilities"": ""Present Fisher..."
2,Circle,circle is a global financial technology firm ...,"Manager, Business Strategy And Operations",4384,"{\n ""Core Responsibilities"": ""Define clear h..."
3,AC Hotel Columbus Downtown,we are hiring a sales manager\n\nsummary\n\nbe...,Sales Manager,2348,"{\n ""Core Responsibilities"": ""Develop and fo..."
4,Nespresso,are you passionate about coffee do you care ab...,Retail Sales Associate,3162,"{\n ""Core Responsibilities"": ""Drive sales, p..."
5,Alberta Health Services,your opportunity\n\ncome join our ach emergenc...,Registered Nurse,2927,"{\n ""Core Responsibilities"": ""Provide nursin..."
6,Professional Placement Services,we are looking for a passionate and skilled co...,Project Manager,1289,"{\n ""Core Responsibilities"": ""Project manage..."
7,FOOD IN MOTION INC.,if you have sales experience in events and can...,Sales And Marketing Specialist,190,"{\n ""Core Responsibilities"": ""Help organize ..."
8,Angott Search Group,myusa credit union million in assets located ...,Human Resources Director,1104,"{\n ""Core Responsibilities"": ""Responsible fo..."
9,Total Quality Logistics,about the role\n\nas a human resources manager...,Human Resources Manager,2313,"{\n ""Core Responsibilities"": ""Serve as HR co..."


In [37]:
dataset_file = 'Extracted_job_Descriptions.csv'
filtered_job_data.to_csv(dataset_file, index=False)



*   **Tokenizing and Embedding Extraction of CV_details and Job Descriptions.**




In [33]:
from transformers import DistilBertTokenizer, DistilBertModel
import torch

# Extracting job descriptions from the "model_response" column
job_descriptions = filtered_job_data['model_response']

# Loading the DistilBERT tokenizer and model
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = DistilBertModel.from_pretrained('distilbert-base-uncased')

# Tokenizing and converting text to embeddings for CVs
cv_embeddings = []

for idx, row in full_cv_extracted.iterrows():
    cv_text = f"JobRole:{row['Job role']} Skills:{row['skills']} Education:{row['Education']}"
    tokens = tokenizer(cv_text, padding=True, truncation=True, return_tensors='pt')
    with torch.no_grad():
        embeddings = model(**tokens).last_hidden_state.mean(dim=1)  # Use mean pooling
    cv_embeddings.append(embeddings)

# Tokenizing and converting text to embeddings for job descriptions
job_embeddings = []
for job_text in job_descriptions:
    tokens = tokenizer(job_text, padding=True, truncation=True, return_tensors='pt')
    with torch.no_grad():
        embeddings = model(**tokens).last_hidden_state.mean(dim=1)  # Use mean pooling
    job_embeddings.append(embeddings)

# Converting lists of embeddings to tensors
cv_embeddings = torch.stack(cv_embeddings)
job_embeddings = torch.stack(job_embeddings)




Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_projector.bias', 'vocab_layer_norm.weight', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [34]:
job_embeddings.shape

torch.Size([14, 1, 768])

In [35]:
cv_embeddings = cv_embeddings.squeeze(dim=1)
job_embeddings = job_embeddings.squeeze(dim = 1)

# 3.Candidate-Job Matching



*   Importing cosine_similarity from sklearn.metrics.




In [36]:
from sklearn.metrics.pairwise import cosine_similarity



*   Calculating cosine similarity between job_embeddings and cv_embeddings.
*   Based on cosine similarity top 5 CVs for each job description is extracted.
*   The extracted top 5 CV pdf names are added to a dataset along with that respective job_description and position title.



In [52]:
cosine_similarities = cosine_similarity(job_embeddings, cv_embeddings)
similarity_df = pd.DataFrame(cosine_similarities, columns=full_cv_extracted['PDF Name'], index=filtered_job_data["position_title"])

# Finding the top 5 CVs for each job description
top_5_cvs = {}
assigned_cvs = set()
for i, job_title in enumerate(similarity_df.index):
    top_matches = similarity_df.iloc[i].nlargest(50)
    filtered_top_matches = [(cv, score) for cv, score in zip(top_matches.index, top_matches.values) if cv not in assigned_cvs][:5]
    assigned_cvs.update(cv for cv, _ in filtered_top_matches)
    # Storing them in the dictionary with appropriate keys
    top_5_cvs[job_title] = {}
    for i, (cv, score) in enumerate(filtered_top_matches, start=1):
        top_5_cvs[job_title][f'Top{i}_CV'] = cv
        top_5_cvs[job_title][f'Top{i}_Score'] = score

# Creating a new dataframe with the top matches
top_5_df = pd.DataFrame(top_5_cvs).transpose()

# Adding the job descriptions as a column
top_5_df.reset_index(inplace=True)
top_5_df.rename(columns={'index': 'Position Title'}, inplace=True)
top_5_df['Job_description'] = filtered_job_data["job_description"]

# Reordering columns
top_5_df = top_5_df[['Job_description','Position Title' ,'Top1_CV', 'Top1_Score', 'Top2_CV', 'Top2_Score', 'Top3_CV', 'Top3_Score', 'Top4_CV', 'Top4_Score', 'Top5_CV', 'Top5_Score']]

# Saving the results to a new CSV file
top_5_df.to_csv('top5_CV_matches.csv', index=False)



*   Displaying Top 5 CV for each job Descriptions with its cosine Similarities.



In [51]:
top_5_df

,Job_description,Position Title,Top1_CV,Top1_Score,Top2_CV,Top2_Score,Top3_CV,Top3_Score,Top4_CV,Top4_Score,Top5_CV,Top5_Score
0,hybrid onsite remote for quasilocal candidate...,Sales leader,24508725.pdf,0.946247,12334140.pdf,0.918244,19124258.pdf,0.912238,11270462.pdf,0.894339,11551946.pdf,0.889806
1,calling all financial advisors looking for a r...,Regional Vice President of Sales,27909372.pdf,0.896222,16605352.pdf,0.894673,14364597.pdf,0.889874,34689720.pdf,0.887807,21773106.pdf,0.88711
2,circle is a global financial technology firm ...,"Manager, Business Strategy And Operations",30504149.pdf,0.888481,36302399.pdf,0.878605,65708020.pdf,0.878314,28697203.pdf,0.875947,13178604.pdf,0.875331
3,we are hiring a sales manager\n\nsummary\n\nbe...,Sales Manager,25561640.pdf,0.883909,15180322.pdf,0.875752,16804396.pdf,0.871957,28198029.pdf,0.871593,91591026.pdf,0.871164
4,are you passionate about coffee do you care ab...,Retail Sales Associate,23185829.pdf,0.883652,34962725.pdf,0.875446,19599699.pdf,0.873622,32636041.pdf,0.872368,25425322.pdf,0.872225
5,your opportunity\n\ncome join our ach emergenc...,Registered Nurse,10333051.pdf,0.896272,12938200.pdf,0.887587,14667957.pdf,0.885179,31292364.pdf,0.884919,52684666.pdf,0.878212
6,we are looking for a passionate and skilled co...,Project Manager,39805617.pdf,0.87809,47919212.pdf,0.877675,27213082.pdf,0.875165,12045067.pdf,0.872903,22349169.pdf,0.8688
7,if you have sales experience in events and can...,Sales And Marketing Specialist,16899268.pdf,0.833561,18176523.pdf,0.832304,13812481.pdf,0.831964,10501991.pdf,0.82865,14771530.pdf,0.828517
8,myusa credit union million in assets located ...,Human Resources Director,39970711.pdf,0.879535,91930382.pdf,0.871561,14107571.pdf,0.870721,27524018.pdf,0.868696,21663616.pdf,0.868568
9,about the role\n\nas a human resources manager...,Human Resources Manager,39081840.pdf,0.892564,19508520.pdf,0.890626,25959103.pdf,0.889145,10694288.pdf,0.887872,13272204.pdf,0.88705




*   Displaying Cosine Similarities



In [54]:
similarity_df.head(4)

PDF Name,15576950.pdf,14640322.pdf,16877897.pdf,15041689.pdf,17855844.pdf,18297650.pdf,11847784.pdf,18731098.pdf,13520837.pdf,14225422.pdf,...,39875803.pdf,98379112.pdf,89197180.pdf,80275976.pdf,81310245.pdf,38007664.pdf,40792022.pdf,77576845.pdf,61677751.pdf,91467795.pdf
position_title,,,,,,,,,,,,,,,,,,,,,
Sales leader,0.813040,0.847913,0.865929,0.827964,0.850069,0.821632,0.810649,0.832111,0.837921,0.846865,...,0.850296,0.861598,0.838345,0.830804,0.851344,0.849483,0.823447,0.846015,0.845229,0.809482
Regional Vice President of Sales,0.834817,0.849110,0.854253,0.844831,0.865172,0.833855,0.830154,0.838806,0.852836,0.860022,...,0.854509,0.871214,0.840751,0.832717,0.843567,0.855598,0.856619,0.855790,0.852268,0.819458
"Manager, Business Strategy And Operations",0.819071,0.848562,0.850159,0.814337,0.858247,0.820382,0.816969,0.815766,0.831635,0.845182,...,0.850267,0.851894,0.826967,0.834050,0.845099,0.860444,0.829734,0.849695,0.844873,0.809905
Sales Manager,0.824277,0.831078,0.840419,0.813744,0.845582,0.804148,0.819553,0.819401,0.839182,0.842709,...,0.846550,0.846794,0.819547,0.823641,0.843782,0.845087,0.842887,0.842976,0.837711,0.787780
